<a href="https://colab.research.google.com/github/Anoushka-Nag/Projects/blob/main/Rectangle%20Colour%20Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import cv2 as cv
import numpy as np
import os


images = []


def detect(image):
    gray = cv.GaussianBlur(image, (7, 7), 0)
    gray = cv.medianBlur(gray, 3)
    gray = cv.cvtColor(gray, cv.COLOR_BGR2GRAY)

    res, imagethreshold = cv.threshold(gray, 200, 255, 0)
    imagethreshold = cv.dilate(imagethreshold, (7, 7), iterations=5)

    imagecontours, hierarchy = cv.findContours(imagethreshold, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)
    for c in imagecontours:
        # if len(imagecontours) > 0 and cv.contourArea(c) > 2000:
        #     cv.drawContours(image, imagecontours, -1, (0, 255, 0), 5)
        peri = cv.arcLength(c, True)
        approx = cv.approxPolyDP(c, 0.02 * peri, True)
        if len(approx) == 4 and cv.contourArea(c) > 2000:
            x, y, w, h = cv.boundingRect(c)
            cv.putText(frame, "Frame Captured", (x, y - 20), cv.FONT_HERSHEY_SIMPLEX, 1.0, (20, 255, 57), 2)
            cv.rectangle(image, (x - 10, y - 10), (x + w + 10, y + h + 10), (171, 71, 0), 2)
            page = image[y:y + h, x:x + w]
            # cv.imshow("page", page)
            images.append(page)
            return True


def find_color(image):
    hsv = cv.cvtColor(image, cv.COLOR_BGR2HSV)

    light_red = np.array([136, 87, 111])
    dark_red = np.array([180, 255, 255])

    light_yellow = np.array([15, 80, 90])
    dark_yellow = np.array([35, 255, 255])

    light_green = np.array([25, 75, 72])
    dark_green = np.array([102, 255, 255])

    light_blue = np.array([94, 80, 2])
    dark_blue = np.array([120, 255, 255])

    mask_red = cv.inRange(hsv, light_red, dark_red)
    
    contours_red, hierarchy = cv.findContours(mask_red, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)
    for c in contours_red:
        cv.drawContours(image, contours_red, -1, (0, 0, 255), 2)

        x, y, w, h = cv.boundingRect(c)
        # cv.rectangle(mask_red, (x, y), (x + w, y + h), (255, 0, 0), 2)
        a_red = w * h
        if a_red >= 1000:
            return "red"

    mask_yellow = cv.inRange(hsv, light_yellow, dark_yellow)
    
    contours_yellow, hierarchy = cv.findContours(mask_yellow, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)
    for c in contours_yellow:
        cv.drawContours(image, contours_yellow, -1, (0, 0, 255), 2)

        x, y, w, h = cv.boundingRect(c)
        # cv.rectangle(mask_yellow, (x, y), (x + w, y + h), (255, 0, 0), 2)
        a_yellow = w * h
        if a_yellow >= 1000:
            return "yellow"

    mask_green = cv.inRange(hsv, light_green, dark_green)
    
    contours_green, hierarchy = cv.findContours(mask_green, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)
    for c in contours_green:
        cv.drawContours(image, contours_green, -1, (0, 0, 255), 2)

        x, y, w, h = cv.boundingRect(c)
        # cv.rectangle(mask_green, (x, y), (x + w, y + h), (255, 0, 0), 2)
        a_green = w * h
        if a_green >= 1000:
            return "green"

    mask_blue = cv.inRange(hsv, light_blue, dark_blue)
    
    contours_blue, hierarchy = cv.findContours(mask_blue, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)
    for c in contours_blue:
        cv.drawContours(image, contours_blue, -1, (0, 0, 255), 2)

        x, y, w, h = cv.boundingRect(c)
        # cv.rectangle(mask_blue, (x, y), (x + w, y + h), (255, 0, 0), 2)
        a_blue = w * h
        if a_blue >= 1000:
            return "blue"
    else:
        return "none"


def save(images):
    i = 1
    for image in images:
        color = find_color(image)
        cv.imwrite(os.path.join("D:/images", "Frame{}_{}.jpg".format(i, color)), image)
        i += 1


cap = cv.VideoCapture(0)
while True:
    isTrue, frame = cap.read()
    seconds = 2
    multiplier = 30 * seconds
    if isTrue:
        frameId = int(round(cap.get(1)))
        if frameId % multiplier == 0:
            if detect(frame):
                cv.imshow("video1", frame)
                cv.waitKey(2000)
        else:
            cv.imshow("video1", frame)

        if cv.waitKey(1) & 0xFF == ord("d"):
            save(images)
            break
    else:
        break
cap.release()
cv.destroyAllWindows()
